In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

C:\Users\yasha\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\yasha\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\yasha\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm  
import tensorflow.keras
import math
from scipy import ndimage

In [3]:
train_dir = "E:\Projects\OCR-Sanskrit\DevanagariHandwrittenCharacterDataset\Train"

In [4]:
!cd DevanagariHandwrittenCharacterDataset/Train/

'list' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
    
    data_format='channels_first',
    validation_split=0.2)

In [12]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(28, 28),
    color_mode = "grayscale",
    batch_size=20,
    shuffle = True,
    classes = ['character_1_ka','character_2_kha','character_3_ga','character_4_gha','character_5_kna','character_6_cha',
               'character_7_chha','character_8_ja','character_9_jha','character_10_yna','character_11_taamatar','character_12_thaa',
               'character_13_daa', 'character_14_dhaa', 'character_15_adna', 'character_16_tabala', 'character_17_tha',
               'character_18_da', 'character_19_dha', 'character_20_na', 'character_21_pa', 'character_22_pha','character_23_ba',
               'character_24_bha','character_25_ma', 'character_26_yaw', 'character_27_ra', 'character_28_la', 'character_29_waw',
               'character_30_motosaw', 'character_31_petchiryakha', 'character_32_patalosaw', 'character_33_ha', 'character_34_chhya', 'character_35_tra', 
               'character_36_gya','digit_0','digit_1','digit_2','digit_3','digit_4','digit_5','digit_6','digit_7','digit_8','digit_9'],
    class_mode="sparse",
    subset='training')

Found 62560 images belonging to 46 classes.


In [13]:
validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(28, 28),
    color_mode = "grayscale",
    batch_size=20,
    shuffle = True,
    classes = ['character_1_ka','character_2_kha','character_3_ga','character_4_gha','character_5_kna','character_6_cha',
               'character_7_chha','character_8_ja','character_9_jha','character_10_yna','character_11_taamatar','character_12_thaa',
               'character_13_daa', 'character_14_dhaa', 'character_15_adna', 'character_16_tabala', 'character_17_tha',
               'character_18_da', 'character_19_dha', 'character_20_na', 'character_21_pa', 'character_22_pha','character_23_ba',
               'character_24_bha','character_25_ma', 'character_26_yaw', 'character_27_ra', 'character_28_la', 'character_29_waw',
               'character_30_motosaw', 'character_31_petchiryakha', 'character_32_patalosaw', 'character_33_ha', 'character_34_chhya', 'character_35_tra', 
               'character_36_gya','digit_0','digit_1','digit_2','digit_3','digit_4','digit_5','digit_6','digit_7','digit_8','digit_9'],
    class_mode="sparse",
    subset='validation') 

Found 15640 images belonging to 46 classes.


In [14]:
from tensorflow import keras

In [15]:
keras.backend.set_image_data_format('channels_first')

In [16]:
from tensorflow import keras
from tensorflow.keras import layers, regularizers

In [17]:
inputs = keras.Input(shape=(1, 28, 28),)

fork11 = layers.Conv2D(64, 5, 5,  activation="relu", padding='same')(inputs)
fork12 = layers.Conv2D(64, 5, 5, activation="relu", padding='same')(inputs)
merge1 = layers.concatenate([fork11, fork12], axis=1, name='merge1')
# concat_feat = concatenate([concat_feat, x], mode='concat', axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))
maxpool1 = layers.MaxPool2D(strides=(2,2), padding='same')(merge1)

fork21 = layers.Conv2D(128, 4, 4, activation="relu", padding='same')(maxpool1)
fork22 = layers.Conv2D(128, 4, 4, activation="relu", padding='same')(maxpool1)
merge2 = layers.concatenate([fork21, fork22, ], axis=1, name='merge2')
maxpool2 = layers.MaxPool2D(strides=(2,2), padding='same')(merge2)

fork31 = layers.Conv2D(256, 3, 3, activation="relu", padding='same')(maxpool2)
fork32 = layers.Conv2D(256, 3, 3, activation="relu", padding='same')(maxpool2)
fork33 = layers.Conv2D(256, 3, 3, activation="relu", padding='same')(maxpool2)
fork34 = layers.Conv2D(256, 3, 3, activation="relu", padding='same')(maxpool2)
fork35 = layers.Conv2D(256, 3, 3, activation="relu", padding='same')(maxpool2)
fork36 = layers.Conv2D(256, 3, 3, activation="relu", padding='same')(maxpool2)
merge3 = layers.concatenate([fork31, fork32, fork33, fork34, fork35, fork36, ], axis=1, name='merge3')
maxpool3 = layers.MaxPool2D(strides=(2,2), padding='same')(merge3)

dropout = layers.Dropout(0.5)(maxpool3)

flatten = layers.Flatten()(dropout)
outputs = layers.Dense(15, activation="softmax")(flatten)

model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1, 28, 28)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 6, 6)     1664        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 6, 6)     1664        input_1[0][0]                    
__________________________________________________________________________________________________
merge1 (Concatenate)            (None, 128, 6, 6)    0           conv2d[0][0]                     
                                                                 conv2d_1[0][0]               

In [18]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.Adadelta(learning_rate = 1.0, rho=0.95),
    metrics = ["accuracy"]
)

In [19]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=100,
                    validation_steps=100,
                    epochs = 15) 

Epoch 1/15
100/100 [==============================] - 182s 2s/step - loss: nan - accuracy: 0.0175 - val_loss: nan - val_accuracy: 0.0245
Epoch 2/15
100/100 [==============================] - 129s 1s/step - loss: nan - accuracy: 0.0240 - val_loss: nan - val_accuracy: 0.0225
Epoch 3/15
100/100 [==============================] - 110s 1s/step - loss: nan - accuracy: 0.0245 - val_loss: nan - val_accuracy: 0.0225
Epoch 4/15
100/100 [==============================] - 91s 914ms/step - loss: nan - accuracy: 0.0215 - val_loss: nan - val_accuracy: 0.0260
Epoch 5/15
100/100 [==============================] - 100s 1000ms/step - loss: nan - accuracy: 0.0230 - val_loss: nan - val_accuracy: 0.0200
Epoch 6/15
100/100 [==============================] - 76s 746ms/step - loss: nan - accuracy: 0.0220 - val_loss: nan - val_accuracy: 0.0175
Epoch 7/15
100/100 [==============================] - 47s 466ms/step - loss: nan - accuracy: 0.0150 - val_loss: nan - val_accuracy: 0.0250
Epoch 8/15
100/100 [===========

### Training accuracy : 83.80% and Validation accuracy : 85.80%

In [12]:
model.save('E:\Projects\HandwrittenEquationSolverCNN\DeepCNN_15EY_sy.h5')